In [14]:
import random
import numpy as np
import pandas as pd
import time
import re
import json
import datetime
import json
import os
import matplotlib.pyplot as plt
import scipy
from scipy import stats

### Sentences

In [45]:
%%time
sdf = pd.read_excel('data/topic_model_data/fullSentDataFrame_8-3.xlsx',index_col=0)
print(len(sdf))
sdf = sdf.drop_duplicates(['sentText'])
print(len(sdf))
sdf.head()

56609
53349
CPU times: user 7.55 s, sys: 1.99 s, total: 9.54 s
Wall time: 24.8 s


,index,newsOutlet,category,dateSeen,url,title,language,sourceCountry,sentText,sentIndexInText,sentTopicID
0,0,apnews.com,western,2021-10-12 07:00:00,https://apnews.com/article/technology-business...,"EU , Ukraine to discuss military training and ...",English,United States,BRUSSELS (AP) — The European Union is consider...,0,-1
1,0,apnews.com,western,2021-10-12 07:00:00,https://apnews.com/article/technology-business...,"EU , Ukraine to discuss military training and ...",English,United States,Acting on a request from Ukraine for help with...,1,-1
2,0,apnews.com,western,2021-10-12 07:00:00,https://apnews.com/article/technology-business...,"EU , Ukraine to discuss military training and ...",English,United States,The topic will be discussed during a summit Tu...,2,-1
3,0,apnews.com,western,2021-10-12 07:00:00,https://apnews.com/article/technology-business...,"EU , Ukraine to discuss military training and ...",English,United States,The results of the mission have yet to be anal...,3,5
4,0,apnews.com,western,2021-10-12 07:00:00,https://apnews.com/article/technology-business...,"EU , Ukraine to discuss military training and ...",English,United States,One official said the EU’s political and secur...,4,-1


In [46]:
import nltk
from nltk import pos_tag, word_tokenize
from nltk.tokenize import sent_tokenize
nltk.download('averaged_perceptron_tagger')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.sentiment.vader import SentiText
nltk.download('vader_lexicon')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/paigelee/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/paigelee/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [47]:
%%time
# add sentiment row to DF
sentiRecs = []
records = sdf.to_dict('records')
for rec in records:
    sid = SentimentIntensityAnalyzer()
    ss = sid.polarity_scores(rec['sentText'])
    artSenti = dict()
    for k in ['pos','neu','neg']:
        artSenti[k] = ss[k]
    sentiRecs.append(artSenti)

CPU times: user 4min 22s, sys: 2.45 s, total: 4min 25s
Wall time: 4min 36s


In [48]:
sentOnlyDf = pd.DataFrame.from_records(sentiRecs)
df = pd.concat([sdf, sentOnlyDf],axis=1)
df['dateSeen'] = df['dateSeen'].apply(lambda d: pd.to_datetime(d))
df['sentiVal'] = df['pos']-df['neg']
df.sample(4)

,index,newsOutlet,category,dateSeen,url,title,language,sourceCountry,sentText,sentIndexInText,sentTopicID,pos,neu,neg,sentiVal
40770,1813.0,tass.com,russian,2022-01-21 08:00:00,https://tass.com/politics/1391611,European security issues shouldnt be limited t...,English,NaN,"I am fairly certain that, one way or another, ...",5.0,12.0,0.045,0.955,0.000,0.045
32731,1295.0,sputniknews.com,russian,2022-01-16 08:00:00,https://sputniknews.com/20220116/some-of-ameri...,Some of America NATO Allies Were Unsettled b...,English,Russia,The draft accords were presented to the US and...,15.0,-1.0,0.149,0.810,0.041,0.108
23662,979.0,sputniknews.com,russian,2022-02-21 08:00:00,https://sputniknews.com/20220221/russia-recogn...,Russia Recognises Donbass Republic Independence,English,Russia,"""We have also heard statements about Ukraine t...",19.0,12.0,0.000,0.726,0.274,-0.274
36254,1488.0,sputniknews.com,russian,2022-02-16 08:00:00,https://sputniknews.com/20220216/cheating-hunt...,New Book on US First Lady Jill May Cause Disco...,English,Russia,Joe Biden has categorically denied the accusat...,16.0,-1.0,0.000,1.000,0.000,0.000


### Write JSON

In [253]:
# jsonDf = df.copy(deep=True)
# jsonDf['dateSeen'] = jsonDf['dateSeen'].apply(lambda d : d.timestamp())
# del jsonDf['month']
# del jsonDf['sourceCountry']

# with open('visualizations/histograms/sentData.json', 'w', encoding='utf-8') as f:
#     json.dump(jsonDf.to_dict('records'), f, ensure_ascii=False, indent=4)

## Reading scores

## Create Topic Dataframe

In [49]:
# TopicID	Topic Keywords	Topic Summary	Western Count	Western Freq	Russian Count	Russian Freq	MoscowTimes Count	MoscowTimes Freq	Total Count
numTopics = 15
topicIDs = range(-1,numTopics)
topicStatData = []
for topicID in topicIDs:
    topicRec = dict()
    topicRec['TopicID'] = topicID
    topicRec['Topic Keywords'] = ''
    topicRec['Topic Summary'] = ''
    
    wdf = df[df['category']=='western']
    rdf = df[df['category']=='russian']
    mtdf = df[df['category']=='moscowtimes']
    
    thiswdf = wdf[wdf['sentTopicID']==topicID]
    thisrdf = rdf[rdf['sentTopicID']==topicID]
    thismtdf = mtdf[mtdf['sentTopicID']==topicID]
  
    topicRec['Western Count'] = len(wdf[wdf['sentTopicID']==topicID])
    topicRec['Russian Count'] = len(rdf[rdf['sentTopicID']==topicID])
    topicRec['MoscowTimes Count'] = len(mtdf[mtdf['sentTopicID']==topicID])
    topicRec['Total Count'] = len(df[df['sentTopicID']==topicID])
    
    topicRec['Freq in Western'] = topicRec['Western Count'] / len(wdf)
    topicRec['Freq in Russian'] = topicRec['Russian Count'] / len(rdf)
    topicRec['Freq in MoscowTimes'] = topicRec['MoscowTimes Count'] / len(mtdf)
    topicRec['Total Freq'] = len(df[df['sentTopicID']==topicID])/len(df)
    
    topicRec['Western Senti Avg'] = thiswdf['pos'].mean()- thiswdf['neg'].mean()
    topicRec['Russian Senti Avg'] = thisrdf['pos'].mean() - thisrdf['neg'].mean()
    topicRec['MoscowTimes Senti Avg'] = thismtdf['pos'].mean()- thismtdf['neg'].mean()
    topicRec['Average Senti'] = df[df['sentTopicID']==topicID]['pos'].mean() - df[df['sentTopicID']==topicID]['neg'].mean()
    
    topicStatData.append(topicRec)
    
topicStats = pd.DataFrame.from_records(topicStatData)
topicStats.sample(6)

,TopicID,Topic Keywords,Topic Summary,Western Count,Russian Count,MoscowTimes Count,Total Count,Freq in Western,Freq in Russian,Freq in MoscowTimes,Total Freq,Western Senti Avg,Russian Senti Avg,MoscowTimes Senti Avg,Average Senti
0,-1,,,9799,11574,1313,22686,0.442393,0.413830,0.406376,0.401188,-0.010843,-0.004378,-0.100167,-0.007394
6,5,,,777,1318,180,2275,0.035079,0.047125,0.055710,0.040232,-0.019656,-0.012105,NaN,-0.014905
10,9,,,194,427,24,645,0.008758,0.015267,0.007428,0.011406,-0.038515,-0.005012,NaN,-0.015478
11,10,,,176,1310,87,1573,0.007946,0.046839,0.026927,0.027818,-0.027409,-0.004763,NaN,-0.007445
3,2,,,470,622,59,1151,0.021219,0.022240,0.018261,0.020355,-0.011215,-0.008264,NaN,-0.009534
15,14,,,2664,2885,277,5826,0.120271,0.103154,0.085732,0.103029,-0.014428,-0.008685,0.084500,-0.011339


In [50]:
topicStats.to_excel('data/topic_model_data/topics_and_sentiment_8-3.xlsx')

### Topic Timeline Json

In [51]:
from datetime import datetime

In [52]:
%%time

totalNumTopics= 15
timeTuples = [
    ("2021-10-01", "2021-10-15"), 
    ("2021-10-16", "2021-10-31"), 
    ("2021-11-01", "2021-11-15"), 
    ("2021-11-16", "2021-11-30"), 
    ("2021-12-01", "2021-12-15"), 
    ("2021-12-16", "2021-12-31"),    
    ("2022-01-01", "2022-01-15"),    
    ("2022-01-16", "2022-01-31"),
    ("2022-02-01", "2022-02-15"),
    ("2022-02-16", "2022-02-28"),]

records = df.to_dict('records')

biweeklyDict = dict()
recCatCounts = dict()
for rec in records:
    if rec['category'] in ['western','russian']:
        # add rec to timerange dict
        for i, (startStr, endStr) in enumerate(timeTuples):
            startDate = datetime.strptime(startStr, '%Y-%m-%d')
            endDate = datetime.strptime(endStr, '%Y-%m-%d')
            if rec['dateSeen'] >= startDate and rec['dateSeen'] <= endDate:
                biweeklyDict.setdefault(i, dict())
                for topicID in range(-1,totalNumTopics):
                    biweeklyDict[i].setdefault(topicID, [])
                biweeklyDict[i][rec['sentTopicID']].append(rec)
                recCatCounts.setdefault(i, {'western' : 0,'russian' : 0})
                recCatCounts[i][rec['category']]+= 1
jsonForTimeline = []
for i in biweeklyDict:
    row = dict()
    row['date'] = timeTuples[i][0].replace('-','')
    row[f'western_ct_period_{i}'] = recCatCounts[i]['western']
    row[f'russian_ct_period_{i}'] = recCatCounts[i]['russian']
    for topicID in range(-1,totalNumTopics):
        row['russian_freq_topic_'+str(topicID)] = len([r for r in biweeklyDict[i][topicID] if r['category']=='russian'])/recCatCounts[i]['russian']
        row['total_freq_topic_'+str(topicID)] = len(biweeklyDict[i][topicID])/(recCatCounts[i]['russian']+recCatCounts[i]['western'])
        row['western_freq_topic_'+str(topicID)] = len([r for r in biweeklyDict[i][topicID] if r['category']=='western'])/recCatCounts[i]['western']
        row['russian_topic_'+str(topicID)] = len([r for r in biweeklyDict[i][topicID] if r['category']=='russian'])
        row['western_topic_'+str(topicID)] = len([r for r in biweeklyDict[i][topicID] if r['category']=='western'])
        row['total_topic_'+str(topicID)] = len(biweeklyDict[i][topicID])
    jsonForTimeline.append(row)

CPU times: user 5.7 s, sys: 244 ms, total: 5.95 s
Wall time: 7.54 s


In [53]:
random.choice(jsonForTimeline)

{'date': '20220116',
 'western_ct_period_7': 3805,
 'russian_ct_period_7': 4640,
 'russian_freq_topic_-1': 0.40323275862068964,
 'total_freq_topic_-1': 0.41420959147424513,
 'western_freq_topic_-1': 0.42759526938239156,
 'russian_topic_-1': 1871,
 'western_topic_-1': 1627,
 'total_topic_-1': 3498,
 'russian_freq_topic_0': 0.039439655172413796,
 'total_freq_topic_0': 0.04227353463587922,
 'western_freq_topic_0': 0.04572930354796321,
 'russian_topic_0': 183,
 'western_topic_0': 174,
 'total_topic_0': 357,
 'russian_freq_topic_1': 0.017456896551724138,
 'total_freq_topic_1': 0.01705150976909414,
 'western_freq_topic_1': 0.016557161629434953,
 'russian_topic_1': 81,
 'western_topic_1': 63,
 'total_topic_1': 144,
 'russian_freq_topic_2': 0.026077586206896552,
 'total_freq_topic_2': 0.023564239194789817,
 'western_freq_topic_2': 0.02049934296977661,
 'russian_topic_2': 121,
 'western_topic_2': 78,
 'total_topic_2': 199,
 'russian_freq_topic_3': 0.008405172413793103,
 'total_freq_topic_3': 0.

In [55]:
import json
with open('visualizations/timeline/freq_data_8-3.json', 'w', encoding='utf-8') as f:
    json.dump(jsonForTimeline, f, ensure_ascii=False, indent=4)

## Co-occurrence analysis

In [56]:
import itertools
import stanza

In [57]:
stanza.download('en')
nlp = stanza.Pipeline('en')

2022-08-03 20:37:31 INFO: Downloading default packages for language: en (English)...
2022-08-03 20:37:33 INFO: File exists: /Users/paigelee/stanza_resources/en/default.zip.
2022-08-03 20:37:37 INFO: Finished downloading models and saved to /Users/paigelee/stanza_resources.
2022-08-03 20:37:37 INFO: Loading these models for language: en (English):
| Processor | Package   |
-------------------------
| tokenize  | combined  |
| pos       | combined  |
| lemma     | combined  |
| depparse  | combined  |
| sentiment | sstplus   |
| ner       | ontonotes |

2022-08-03 20:37:37 INFO: Use device: cpu
2022-08-03 20:37:37 INFO: Loading: tokenize
2022-08-03 20:37:38 INFO: Loading: pos
2022-08-03 20:37:41 INFO: Loading: lemma
2022-08-03 20:37:41 INFO: Loading: depparse
2022-08-03 20:37:42 INFO: Loading: sentiment
2022-08-03 20:37:43 INFO: Loading: ner
2022-08-03 20:37:45 INFO: Done loading processors!


In [58]:
import nltk
from nltk.corpus import stopwords

engStopwords = set(stopwords.words('english'))

In [59]:
jsonDict = dict()

In [ ]:
%%time
# get cooccurrence connections
thresh = 5
n = 4

allSentRecords = sdf.to_dict('records')

for cat in ['russian']:
    networkJson = dict()
    networkJson['nodes'] = []
    networkJson['links'] = []

    curatedNodes = set()
    linkCounter = dict()
    keywordCounter = dict()

    windowList = []
    records = []
    for d in allSentRecords:
        if d['category'] == cat and d['sentTopicID'] != -1:
            records.append(d)
    for i, rec in enumerate(records):
        if i % 1000 == 0:
            print(f'{i}/{len(records)} recs parsed by NLP')
        docText = rec['sentText']
        nlpdoc = nlp(docText)
        wordList = [w for w in nlpdoc.iter_words() if w.text.isalpha() and w.text.lower() not in engStopwords]
        for windowIdx in range(len(wordList) // n + len(wordList) % n):
            startIdx = windowIdx
            endIdx = windowIdx+random.choice(list(range(1,n+1)))
            wordWindow = wordList[startIdx:endIdx]
            tokens = []
            for wordToken in wordWindow:
                formattedToken = wordToken.lemma.lower()
                if formattedToken in engStopwords:
                    continue
                tokens.append(formattedToken)
            windowList.append(tokens)
        for t1 in [w.lemma.lower() for w in wordList]:
            keywordCounter.setdefault(t1, 0)
            keywordCounter[t1] += 1
    print('\tdone nlping docs')
    sufficientNodes = set()
    for key in keywordCounter:
        if keywordCounter[key] >= thresh:
            sufficientNodes.add(key)
    print('\tfound sufficient recs')
    for tokens in windowList:
        s = set(tokens)
        subsets = [tuple(i) for i in itertools.combinations(s, 2)]
        for t1, t2 in subsets:
            if t1 != t2 and t1 in sufficientNodes and t2 in sufficientNodes:
                label = '_'.join(sorted([t1, t2]))
                linkCounter.setdefault(label, 0)
                linkCounter[label] += 1
    print('\tgot linked tokens')
    node2id = dict()
    for i, node in enumerate(sufficientNodes):
        networkJson['nodes'].append({
            'id' : node,
            'occurrences' : keywordCounter[node]
        })
        node2id[node] = i

    for label in linkCounter:
        source, target = label.split('_')
        networkJson['links'].append({
            'source' : node2id[source],
            'target' : node2id[target],
            'value' : linkCounter[label]
        })
    
    jsonDict[cat] = networkJson
    
    with open(f'visualizations/ngramsNetwork/{cat}ngramNet{n}_8-4.json', 'w', encoding ='ascii') as json_file:
        json.dump(networkJson, json_file, ensure_ascii = True)
    print('\twrote json')

0/16394 recs parsed by NLP
1000/16394 recs parsed by NLP
2000/16394 recs parsed by NLP


In [ ]:
1

In [63]:
with open(f'visualizations/ngramsNetwork/{cat}ngramNet{n}_8-4.json', 'w', encoding ='ascii') as json_file:
    json.dump(networkJson, json_file, ensure_ascii = True)

### Article

In [11]:
# df = pd.read_excel('data/topic model data/topicArticleFrame7-31.xlsx',index_col=0)
# df.sample(5)

FileNotFoundError: [Errno 2] No such file or directory: 'data/topic model data/topicArticleFrame7-31.xlsx'

In [ ]:
# TopicID	Topic Keywords	Topic Summary	Western Count	Western Freq	Russian Count	Russian Freq	MoscowTimes Count	MoscowTimes Freq	Total Count
topicCols = [col for col in df.columns if 'topic' in col]
topicStatData = []
for topicCol in topicCols:
    topicRec = dict()
    topicRec['TopicID'] = int(topicCol.split('_')[1])
    topicRec['Topic Keywords'] = ''
    topicRec['Topic Summary'] = ''
    
    topicRec['Western Count'] = df[df['category']=='western'][topicCol].sum()
    topicRec['Freq in Western'] = df[df['category']=='western'][topicCol].sum()/df[df['category']=='western']['sentsInArticle'].sum()
    
    topicRec['Russian Count'] = df[df['category']=='russian'][topicCol].sum()
    topicRec['Freq in Russian'] = df[df['category']=='russian'][topicCol].sum()/df[df['category']=='russian']['sentsInArticle'].sum()
    
    topicRec['MoscowTimes Count'] = df[df['category']=='moscowtimes'][topicCol].sum()
    topicRec['Freq in MoscowTimes'] = df[df['category']=='moscowtimes'][topicCol].sum()/df[df['category']=='moscowtimes']['sentsInArticle'].sum()
    
    topicRec['Total Count'] = df[topicCol].sum()
    topicRec['Total Freq'] = df[topicCol].sum()/df['sentsInArticle'].sum()
    topicStatData.append(topicRec)
    
topicStats = pd.DataFrame.from_records(topicStatData)
topicStats.sample(6)

,TopicID,Topic Keywords,Topic Summary,Western Count,Freq in Western,Russian Count,Freq in Russian,MoscowTimes Count,Freq in MoscowTimes,Total Count,Total Freq
20,19,,,469,0.018757,674,0.022437,22,0.006681,1165,0.019971
13,12,,,792,0.031675,1289,0.042911,148,0.044944,2229,0.038210
11,10,,,138,0.005519,629,0.020939,56,0.017006,823,0.014108
15,14,,,268,0.010718,353,0.011751,148,0.044944,769,0.013182
18,17,,,842,0.033675,699,0.023270,90,0.027331,1631,0.027959
17,16,,,375,0.014998,79,0.002630,49,0.014880,503,0.008622
